In [ ]:
###################################
# new_field_name_list

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
['eBay', 'item', 'number:371252240431', 'Seller', 'assumes', 'all', 'responsibility', 'for', 'this', 'listing', '.', 'Last', 'updated', 'on', 'May', '04', ',', '2015', '10:34:20', 'PDT', 'View', 'all', 'revisions', 'Item', 'specifics', 'Condition', ':', 'New', ':', 'A', 'brand-new', ',', 'unused', ',', 'unopened', ',', 'undamaged', 'item', 'in', 'its', 'original', 'packaging', '(', 'where', 'packaging', 'is', 'applicable', ')', '.', 'Packaging', 'should', 'be', 'the', 'same', 'as', 'what', 'is', 'found', 'in', 'a', 'retail', 'store', ',', 'unless', 'the', 'item', 'is', 'handmade', 'or', 'was', 'packaged', 'by', 'the', 'manufacturer', 'in', 'non-retail', 'packaging', ',', 'such', 'as', 'an', 'unprinted', 'box', 'or', 'plastic', 'bag', '.', 'See', 'the', 'seller', "'s", 'listing', 'for', 'full', 'details', '.', 'See', 'all', 'condition', 'definitions-', 'opens', 'in', 'a', 'new'

##########################################################################

This is version 1 of trying to modularize code so that we can run many experiments and collaborate on this colab.
The functions that are common to all experiments that need to be run first are at the bottom of the document beginning with a line of #'s like below:
##########################################################################

#SETUP TO RUN CODE

##IMPORTS AND PIP INSTALLS

In [1]:
#pip installs
! pip install ipdb -q
import ipdb 
#place the line of code below from whichever line you want to start debugging
#type n for step over and s for step into
# ipdb.set_trace()

from time import time
import gensim
import pandas as pd
from typing import List, Tuple, Callable
import numpy as np
from gensim.models.keyedvectors import KeyedVectors
from gensim.models.wrappers import FastText
from gensim.similarities import WmdSimilarity
from ast import literal_eval

##MOUNT GOOGLE DRIVE SO THAT WE CAN ACCESS FILES FROM GOOGLE DRIVE

In [2]:
#repoPath should start with the path 
#'/content/gdrive/My Drive/' and go to the folder where the git repo is, and should end with the name of the repo:
repoPath = '/content/gdrive/My Drive/TestingGitCloneIntoGoogleDrive/textSimilarityUsingWordMoversDistance'

from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
import os
os.chdir(repoPath)

Mounted at /content/gdrive


#DOWNLOAD FILES TO data/ FOLDER (1 time setup)

##Input

In [3]:
##############################
#centralized data folder
##############################
#centralized folder for data
folder_for_data_with_slash_at_end = 'data/'

##############################
#product dataset related inputs
##############################
#url of csv dataset to download
url_of_dataset = 'https://query.data.world/s/iccclhgzbyedtv54p5lresglnun3qz'

#desired filename of dataset
filename_of_original_dataset = 'crowdflower_product_dataset.csv'

##############################
#word embedding related inputs
##############################
#pretrained word embeddings to load
# url_to_word_embedding_zip = "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"
url_to_word_embedding_zip = "https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.en.zip"


##Code to download data to data folder

In [8]:
original_dataset_path = folder_for_data_with_slash_at_end+filename_of_original_dataset

#############################################
#Download product dataset
def download_csv_from_url_to_local_directory(url_of_dataset: str, desired_path_of_csv_to_be_created: str, csv_encoding: str = "ISO-8859-1") -> bool:
  import os.path
  from os import path
  import pandas as pd
  if path.exists(desired_path_of_csv_to_be_created) == True:
    print('csv already exists at path')
    success_in_downloading_dataset = True 
  else:
    product_dataset = pd.read_csv(url_of_dataset, encoding = csv_encoding)
    product_dataset.to_csv(desired_path_of_csv_to_be_created, index = False)
    if path.exists(desired_path_of_csv_to_be_created) == True:
      success_in_downloading_dataset = True
    else:
      success_in_downloading_dataset = False
      print('Failed to download product dataset: ', url_of_dataset)
  return success_in_downloading_dataset



success_product_dataset_download = download_csv_from_url_to_local_directory(url_of_dataset, original_dataset_path)



#############################################
##download model
# folder_for_data_with_slash_at_end = "data/"
download_success, word_embedding_file_path = download_and_unzip_pretrained_word_embedding(url_to_word_embedding_zip, folder_for_data_with_slash_at_end)


##############################################




csv already exists at path
--2020-06-23 03:50:15--  https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.en.zip
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.75.142, 172.67.9.4, 104.22.74.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.75.142|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.

Archive:  data/wiki.en.zip
  inflating: data/wiki.en.vec        

#Experiment Setup:
Creates class that holds experiment related variables that will be used throughout experiment. 

Output:

-sampled product dataframe csv file

-queried products dataframe csv file

##Input

In [9]:
##############################
#centralized data folder
##############################
folder_for_data_with_slash_at_end = 'data/'


##############################
#experiment folder related inputs
##############################
experiment_output_folder_path = 'experiment_outputs/experiment5/'


##############################
#product dataset related inputs
##############################
#dataset to load
filename_of_original_dataset = 'crowdflower_product_dataset.csv'
list_of_indexes_to_sample_from_original_dataset = [i for i in range(0,500)]


#sampled product dataframe csv name
#path initializations
sampled_product_dataframe_csv_name = 'sampled_product_dataframe.csv'

##############################
##Create Query Products List
##############################
#queried products file path
queried_products_desired_csv_path = 'queried_products_sample_dataframe.csv'

##Code

In [10]:
class ProductSimilarityExperiment:
    def __init__(self, folder_for_data_with_slash_at_end,  
                 filename_of_original_dataset, 
                 list_of_indexes_to_sample_from_original_dataset,
                 experiment_output_folder_path,
                 sampled_product_dataframe_csv_name,
                 queried_products_desired_csv_path
                 ):
       
        ##############################
        #centralized data folder
        ##############################
        #centralized folder for data
        self.folder_for_data_with_slash_at_end = folder_for_data_with_slash_at_end

        ##############################
        #experiment folder related inputs
        ##############################
        #folder path for experiment
        self.experiment_output_folder_path = experiment_output_folder_path

        ##############################
        #product dataset related inputs
        ##############################
        #filename of dataset
        self.original_dataset_path = folder_for_data_with_slash_at_end + filename_of_original_dataset

        #list of indexes to sample from original dataset
        self.list_of_indexes_to_sample_from_original_dataset = list_of_indexes_to_sample_from_original_dataset

        #sampled product dataframe csv name
        self.sampled_product_dataframe_csv_path = experiment_output_folder_path + sampled_product_dataframe_csv_name

        #queried products file path
        self.queried_products_sample_dataframe_csv_path = experiment_output_folder_path+queried_products_desired_csv_path

        ##############################
        #preprocessing related inputs
        ##############################
        self.list_of_preprocessed_text_fields = [] #this is new_field_name_list
        self.product_dataframe_with_preprocessed_text_csv_file = ''
        self.product_dataframe_with_preprocessed_text_csv_path = ''
        self.dictionary_of_converters_for_read_csv = {}

        #############################################
        #create folder for experiment
        import os.path
        from os import path
        if path.exists(experiment_output_folder_path) == True:
          print("Folder already exists, so folder creation failed")
        else:
          !mkdir $experiment_output_folder_path

        ##############################
        #initialization functions
        ##############################
        product_dataframe = load_csv_from_local_directory_to_dataframe(self.original_dataset_path)
        product_dataframe = product_dataframe.loc[self.list_of_indexes_to_sample_from_original_dataset,:]
        product_dataframe.to_csv(self.sampled_product_dataframe_csv_path, index=False)



    def create_queried_products_csv_file_by_sampling_n_values_from_each_query(
        self, 
        number_of_products_per_query_to_sample_from_sampled_product_dataframe,
        randomly_sample_from_each_query = False
        ):
      sampled_product_dataframe = load_csv_from_local_directory_to_dataframe(self.sampled_product_dataframe_csv_path)

      queried_products_sample_dataframe = sample_n_products_from_all_query_fields_in_dataframe(sampled_product_dataframe, number_of_products_per_query_to_sample_from_sampled_product_dataframe, randomly_sample_from_each_query)
      queried_products_sample_dataframe.to_csv(self.queried_products_sample_dataframe_csv_path, index=False)

    def create_queried_products_csv_file_by_giving_index_list_of_desired_queries(
        self, 
        list_of_indexes_of_original_product_dataframe_to_use_as_queried_products,
        randomly_sample_from_each_query = False
        ):
      sampled_product_dataframe = load_csv_from_local_directory_to_dataframe(self.sampled_product_dataframe_csv_path)

      queried_products_sample_dataframe = sample_n_products_from_all_query_fields_in_dataframe(sampled_product_dataframe, self.number_of_products_per_query_to_sample, self.randomly_sample_from_each_query)
      queried_products_sample_dataframe.to_csv(self.queried_products_sample_dataframe_csv_path, index=False)


In [11]:

##############################
##Setup Parameters and datafiles commonly used for experiment
##############################
experiment = ProductSimilarityExperiment(
    folder_for_data_with_slash_at_end = folder_for_data_with_slash_at_end, 
    filename_of_original_dataset = filename_of_original_dataset, 
    list_of_indexes_to_sample_from_original_dataset = list_of_indexes_to_sample_from_original_dataset,
    experiment_output_folder_path = experiment_output_folder_path,
    sampled_product_dataframe_csv_name = sampled_product_dataframe_csv_name,
    queried_products_desired_csv_path = queried_products_desired_csv_path
    )

##############################
##Create Query Products List
##############################
number_of_products_per_query_to_sample_from_sampled_product_dataframe = 2
randomly_sample_from_each_query = False
experiment.create_queried_products_csv_file_by_sampling_n_values_from_each_query(number_of_products_per_query_to_sample_from_sampled_product_dataframe = 2, randomly_sample_from_each_query = False)


Folder already exists, so folder creation failed


#Preprocessing Text Function

##INPUT

In [12]:
#preprocessing file path
product_dataframe_with_preprocessed_text_csv_file = 'product_dataframe_with_preprocessed_text.csv'

#which preprocessing to use
list_of_fields_to_preprocess_in_order_of_appending = [['product_title'], ['product_description'], ['product_title', 'product_description'] ]
# list_of_fields_to_preprocess_in_order_of_appending = [['product_title']]


#choose list of preprocessing function with each element corresponding to same index field in list_of_fields_to_preprocess_in_order_of_appending
preprocess_data_object = PreprocessData()
preprocessing_function = preprocess_data_object.text_preprocessing_for_WMD
list_of_preprocessing_functions_for_each_field_in_order_of_appending = [preprocessing_function,preprocessing_function,preprocessing_function]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


##Code

In [13]:
def preprocess_text_from_dataframe_fields_and_output_csv(
    product_similarity_experiment_object, list_of_fields_to_preprocess_in_order_of_appending, 
    list_of_preprocessing_functions_for_each_field_in_order_of_appending, 
    product_dataframe_with_preprocessed_text_csv_file
    ):

  product_dataframe_with_preprocessed_text_csv_path = experiment_output_folder_path+product_dataframe_with_preprocessed_text_csv_file
  #below new_field_name is initialized based on list_of_fields_to_preprocess_in_order_of_appending
  list_of_preprocessed_text_fields = []
  for input_fields_list in list_of_fields_to_preprocess_in_order_of_appending:
    new_field_name = "_AND_".join(input_fields_list)+'_preprocessed'
    list_of_preprocessed_text_fields.append(new_field_name)
  dictionary_of_converters_for_read_csv={}
  for i in range(len(list_of_preprocessed_text_fields)):
    dict_key = list_of_preprocessed_text_fields[i]
    dictionary_of_converters_for_read_csv[dict_key] = eval


  #############################################
  # #preprocess the input dataset text. Save the preprocessing
  exp_obj = product_similarity_experiment_object
  product_dataframe = load_csv_from_local_directory_to_dataframe(exp_obj.sampled_product_dataframe_csv_path)
  preprocess_data_object = PreprocessData()

  # list_of_fields_to_preprocess_in_order_of_appending = [['product_title'], ['product_description'], ['product_title', 'product_description'] ]
  product_dataframe_with_preprocessed_text = product_dataframe
  for i in range(len(list_of_fields_to_preprocess_in_order_of_appending)):
    input_fields_list = list_of_fields_to_preprocess_in_order_of_appending[i]
    preprocessing_function = list_of_preprocessing_functions_for_each_field_in_order_of_appending[i]
    # preprocessing_function = preprocess_data_object.text_preprocessing_for_WMD
    product_dataframe_with_preprocessed_text = preprocess_data_object.create_new_field_in_dataframe_with_preprocessed_text_from_list_of_existing_fields(product_dataframe_with_preprocessed_text, input_fields_list, list_of_preprocessed_text_fields[i], preprocessing_function)

  product_dataframe_with_preprocessed_text.to_csv(product_dataframe_with_preprocessed_text_csv_path, index=False)
  return list_of_preprocessed_text_fields, dictionary_of_converters_for_read_csv

list_of_preprocessed_text_fields, dictionary_of_converters_for_read_csv = preprocess_text_from_dataframe_fields_and_output_csv(
    product_similarity_experiment_object = experiment, 
    list_of_fields_to_preprocess_in_order_of_appending = list_of_fields_to_preprocess_in_order_of_appending, 
    list_of_preprocessing_functions_for_each_field_in_order_of_appending = list_of_preprocessing_functions_for_each_field_in_order_of_appending, 
    product_dataframe_with_preprocessed_text_csv_file = product_dataframe_with_preprocessed_text_csv_file
    )
experiment.list_of_preprocessed_text_fields = list_of_preprocessed_text_fields #this is new_field_name_list
experiment.product_dataframe_with_preprocessed_text_csv_file = product_dataframe_with_preprocessed_text_csv_file
experiment.product_dataframe_with_preprocessed_text_csv_path = experiment.experiment_output_folder_path + experiment.product_dataframe_with_preprocessed_text_csv_file
experiment.dictionary_of_converters_for_read_csv = dictionary_of_converters_for_read_csv
  

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#Base Similarity Score Creation

##Input

In [14]:
# #similarity_matrix_path
# word_embedding_file_name = 'GoogleNews-vectors-negative300.bin'
# word_embedding_used = 'word2vec'
# similarity_measure_type = 'wmd'

################################
# word_embedding_used = 'word2vec'
# word_embedding_file_name = 'GoogleNews-vectors-negative300.bin'

word_embedding_used = 'fasttext'
word_embedding_file_name = "wiki.en.bin"


similarity_measure_type = 'cosine_similarity'

##Code

###Load model separately because it takes a long time


In [15]:
# #load mode
# word_embedding_file_path = experiment.folder_for_data_with_slash_at_end + word_embedding_file_name
# # #load any pretrained word embedding models
# word2vec_model = load_word2vec_word_embedding_into_KeyedVectors_model(word_embedding_file_path)
# # model = load_fasttext_word_embedding_into_KeyedVectors_model(word_embedding_file_path)


###########################
word_embedding_file_path = experiment.folder_for_data_with_slash_at_end + word_embedding_file_name
if word_embedding_used == 'word2vec':
    model = KeyedVectors.load_word2vec_format(word_embedding_file_path, binary=True)
elif word_embedding_used == 'fasttext':
    from gensim.models.wrappers import FastText
    model = FastText.load_fasttext_format(word_embedding_file_path)

In [16]:
# def create_base_similarity_dataframe_csv_for_each_preprocessing_field_using_Word2Vec_and_WMD(
#   experiment,
#   word_embedding_file_name,
#   word_embedding_used, 
#   similarity_measure_type,
#   word2vec_model
#   ):
  
#   exp = experiment

#   similarity_dataframe_csv_path_list = []
#   for preprocessing_used in exp.list_of_preprocessed_text_fields:
#     list_of_text_to_combine_into_one_path_for_similarity_dataframe  = [exp.experiment_output_folder_path+'similarity_dataframe', preprocessing_used, word_embedding_used, similarity_measure_type,'.csv']
#     similarity_dataframe_csv_path = "_".join(list_of_text_to_combine_into_one_path_for_similarity_dataframe)
#     similarity_dataframe_csv_path_list.append(similarity_dataframe_csv_path)
#   # print(list_of_preprocessed_text_fields)
#   # print(similarity_dataframe_csv_path_list)


#   word_embedding_file_path = experiment.folder_for_data_with_slash_at_end + word_embedding_file_name
#   print("word_embedding_file_path")
#   ###################
#   #############################################
#   #create a similarity model that I can query

#   for i in range(len(exp.list_of_preprocessed_text_fields)):
#     print("i = ", i)
#     preprocessing_used = exp.list_of_preprocessed_text_fields[i]
#     similarity_dataframe_csv_path = similarity_dataframe_csv_path_list[i]

#     #create corpus of words to calculate similarity from
#     product_dataframe_with_preprocessed_text = pd.read_csv(exp.product_dataframe_with_preprocessed_text_csv_path, converters=exp.dictionary_of_converters_for_read_csv)
#     field_name_of_selected_preprocessed_text_to_create_corpus_from = preprocessing_used
#     # ipdb.set_trace()
#     corpus = product_dataframe_with_preprocessed_text[field_name_of_selected_preprocessed_text_to_create_corpus_from].tolist()



#     wmd_object_for_description = WmdSimilarity(corpus, word2vec_model)
#     # print("loaded wmd object")

#     #############################################
#     #Loop through all list of queries
#     # product_dataframe_with_preprocessed_text = load_csv_from_local_directory_to_dataframe(product_dataframe_with_preprocessed_text_csv_path)
#     product_dataframe_with_preprocessed_text = pd.read_csv(exp.product_dataframe_with_preprocessed_text_csv_path, converters=exp.dictionary_of_converters_for_read_csv)
#     field_name_of_selected_preprocessed_text_to_create_corpus_from = preprocessing_used
#     queried_products_sample_dataframe = load_csv_from_local_directory_to_dataframe(exp.queried_products_sample_dataframe_csv_path)
#     num_rows_queried_products_sample_dataframe = queried_products_sample_dataframe.shape[0]

#     # similarity_dataframe = pd.DataFrame(columns=['searched_item_unit_id', 'similar_item_unit_id', 'similarity_score'])
#     similarity_dictionary = {
#         'searched_item_unit_id': [],
#         'similar_item_unit_id': [],
#         'similarity_score': []
#     }
#     for i in range(num_rows_queried_products_sample_dataframe):
#       queried_product_index = queried_products_sample_dataframe['_unit_id'][i]
#       # ipdb.set_trace()
#       queried_product_preprocessed_text = product_dataframe_with_preprocessed_text.loc[product_dataframe_with_preprocessed_text['_unit_id'] == queried_product_index][field_name_of_selected_preprocessed_text_to_create_corpus_from].iloc[0]
#       # print(queried_product_preprocessed_text)
#       # queried_product_preprocessed_text = queried_products_sample_dataframe[field_name_of_selected_preprocessed_text_to_create_corpus_from][i]
#       similar_item_indexes_and_similarity = wmd_object_for_description[queried_product_preprocessed_text]
#       # print(similar_item_indexes_and_similarity)
#       searched_item_unit_id = queried_products_sample_dataframe['_unit_id'][i]
#       # ipdb.set_trace()
#       for j in range(len(similar_item_indexes_and_similarity)):
#         similar_item_index = j
#         similar_item_unit_id = product_dataframe_with_preprocessed_text['_unit_id'][similar_item_index]
#         similarity_score = similar_item_indexes_and_similarity[j]
#         # dataframe_row_we_are_appending = pd.DataFrame(data = {'searched_item_unit_id': [searched_item_unit_id], 'similar_item_unit_id': [similar_item_unit_id], 'similarity_score': [similarity_score]})
#         similarity_dictionary['searched_item_unit_id'].append(searched_item_unit_id)
#         similarity_dictionary['similar_item_unit_id'].append(similar_item_unit_id)
#         similarity_dictionary['similarity_score'].append(similarity_score)
#         # ipdb.set_trace()
#         # similarity_dataframe.append(dataframe_row_we_are_appending, ignore_index=True)

#     similarity_dataframe = pd.DataFrame(data=similarity_dictionary)
#     similarity_dataframe.to_csv(similarity_dataframe_csv_path, index=False)

# create_base_similarity_dataframe_csv_for_each_preprocessing_field_using_Word2Vec_and_WMD(
#   experiment = experiment,
#   word_embedding_file_name = word_embedding_file_name,
#   word_embedding_used = word_embedding_used, 
#   similarity_measure_type = similarity_measure_type,
#   word2vec_model = word2vec_model
#   )

###############################
# !python -c "from DocSim import DocSim"
# from DocSim import DocSim
ds = DocSim(model)

#############################

def create_base_similarity_dataframe_csv_for_each_preprocessing_field_using_cosine_similarity(
  experiment,
  word_embedding_file_name,
  word_embedding_used, 
  similarity_measure_type,
  model
  ):
  
  exp = experiment

  similarity_dataframe_csv_path_list = []
  for preprocessing_used in exp.list_of_preprocessed_text_fields:
    list_of_text_to_combine_into_one_path_for_similarity_dataframe  = [exp.experiment_output_folder_path+'similarity_dataframe', preprocessing_used, word_embedding_used, similarity_measure_type,'.csv']
    similarity_dataframe_csv_path = "_".join(list_of_text_to_combine_into_one_path_for_similarity_dataframe)
    similarity_dataframe_csv_path_list.append(similarity_dataframe_csv_path)
  # print(list_of_preprocessed_text_fields)
  # print(similarity_dataframe_csv_path_list)


  word_embedding_file_path = experiment.folder_for_data_with_slash_at_end + word_embedding_file_name
  print("word_embedding_file_path")
  ###################
  #############################################
  #create a similarity model that I can query

  for i in range(len(exp.list_of_preprocessed_text_fields)):
    print("i = ", i)
    preprocessing_used = exp.list_of_preprocessed_text_fields[i]
    similarity_dataframe_csv_path = similarity_dataframe_csv_path_list[i]

    #create corpus of words to calculate similarity from
    product_dataframe_with_preprocessed_text = pd.read_csv(exp.product_dataframe_with_preprocessed_text_csv_path, converters=exp.dictionary_of_converters_for_read_csv)
    field_name_of_selected_preprocessed_text_to_create_corpus_from = preprocessing_used
    # ipdb.set_trace()
    corpus = product_dataframe_with_preprocessed_text[field_name_of_selected_preprocessed_text_to_create_corpus_from].tolist()


    ##!!!!!!!!!!!!!!
    # wmd_object_for_description = WmdSimilarity(corpus, word2vec_model)
    # # print("loaded wmd object")

    #############################################
    #Loop through all list of queries
    # product_dataframe_with_preprocessed_text = load_csv_from_local_directory_to_dataframe(product_dataframe_with_preprocessed_text_csv_path)
    product_dataframe_with_preprocessed_text = pd.read_csv(exp.product_dataframe_with_preprocessed_text_csv_path, converters=exp.dictionary_of_converters_for_read_csv)
    field_name_of_selected_preprocessed_text_to_create_corpus_from = preprocessing_used
    queried_products_sample_dataframe = load_csv_from_local_directory_to_dataframe(exp.queried_products_sample_dataframe_csv_path)
    num_rows_queried_products_sample_dataframe = queried_products_sample_dataframe.shape[0]

    # similarity_dataframe = pd.DataFrame(columns=['searched_item_unit_id', 'similar_item_unit_id', 'similarity_score'])
    similarity_dictionary = {
        'searched_item_unit_id': [],
        'similar_item_unit_id': [],
        'similarity_score': []
    }
    for i in range(num_rows_queried_products_sample_dataframe):
      queried_product_index = queried_products_sample_dataframe['_unit_id'][i]
      # ipdb.set_trace()
      queried_product_preprocessed_text = product_dataframe_with_preprocessed_text.loc[product_dataframe_with_preprocessed_text['_unit_id'] == queried_product_index][field_name_of_selected_preprocessed_text_to_create_corpus_from].iloc[0]
      # print(queried_product_preprocessed_text)
      # queried_product_preprocessed_text = queried_products_sample_dataframe[field_name_of_selected_preprocessed_text_to_create_corpus_from][i]
      
      ##!!!!!!!!!!!!!!
      # similar_item_indexes_and_similarity = wmd_object_for_description[queried_product_preprocessed_text]
      # # print(similar_item_indexes_and_similarity)

      searched_item_unit_id = queried_products_sample_dataframe['_unit_id'][i]
      # ipdb.set_trace()
      for j in range(product_dataframe_with_preprocessed_text.shape[0]):
        similar_item_index = j
        similar_item_unit_id = product_dataframe_with_preprocessed_text['_unit_id'][similar_item_index]
        
        ##!!!!!!!!!!!!!!
        # similarity_score = similar_item_indexes_and_similarity[j]
        compared_product_preprocessed_text = product_dataframe_with_preprocessed_text.loc[j, :][field_name_of_selected_preprocessed_text_to_create_corpus_from]
        # ipdb.set_trace()

        output = ds.calculate_similarity_mod(queried_product_preprocessed_text, compared_product_preprocessed_text)
        if output != []:
          similarity_score = output[0]["score"]
        else:
          similarity_score = 0
        
        
        # dataframe_row_we_are_appending = pd.DataFrame(data = {'searched_item_unit_id': [searched_item_unit_id], 'similar_item_unit_id': [similar_item_unit_id], 'similarity_score': [similarity_score]})
        similarity_dictionary['searched_item_unit_id'].append(searched_item_unit_id)
        similarity_dictionary['similar_item_unit_id'].append(similar_item_unit_id)
        similarity_dictionary['similarity_score'].append(similarity_score)
        # ipdb.set_trace()
        # similarity_dataframe.append(dataframe_row_we_are_appending, ignore_index=True)

    similarity_dataframe = pd.DataFrame(data=similarity_dictionary)
    similarity_dataframe.to_csv(similarity_dataframe_csv_path, index=False)

create_base_similarity_dataframe_csv_for_each_preprocessing_field_using_cosine_similarity(
  experiment = experiment,
  word_embedding_file_name = word_embedding_file_name,
  word_embedding_used = word_embedding_used, 
  similarity_measure_type = similarity_measure_type,
  model = model
  )



word_embedding_file_path
i =  0
i =  1


/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


i =  2


#Composite Similarity Score Creation

##Input

In [23]:
###########################################
#list of similarity dataframes
#input
list_of_files_in_experiment_output_folder_of_base_similarity_measure_csv_files = \
['similarity_dataframe_product_title_preprocessed_fasttext_cosine_similarity_.csv',
'similarity_dataframe_product_description_preprocessed_fasttext_cosine_similarity_.csv']


###########################################
composite_similarity_dataframe_csv_name = 'composite_similarity_dataframe.csv'

###########################################
relevance_threshold = 2


###########################################
num_most_similar_items_for_accuracy = 10


###########################################
#composite function of all the similarity scores
def composite_similarity_function(input_base_similarity_scores: List[float])-> float:
  return input_base_similarity_scores[0]*0.5+input_base_similarity_scores[1]*0.5


##Code

In [24]:
def create_composite_similarity_dataframe_csv_with_assuming_query_is_available(
  experiment,
  list_of_files_in_experiment_output_folder_of_base_similarity_measure_csv_files,
  composite_similarity_dataframe_csv_name, 
  relevance_threshold, 
  num_most_similar_items_for_accuracy, 
  composite_similarity_function):


  #experiment folder path
  #input
  exp = experiment

  experiment_output_folder_path = exp.experiment_output_folder_path

  queried_products_sample_dataframe_csv_path = exp.queried_products_sample_dataframe_csv_path

  sampled_product_dataframe_csv_path = exp.sampled_product_dataframe_csv_path

  composite_similarity_dataframe_csv_path = exp.experiment_output_folder_path + 'composite_similarity_dataframe.csv'

  ######################

  #calculated based on input
  list_of_paths_in_experiment_output_folder_of_base_similarity_measure_csv_files = \
  [experiment_output_folder_path+base_similarity for base_similarity in list_of_files_in_experiment_output_folder_of_base_similarity_measure_csv_files]
  print(list_of_paths_in_experiment_output_folder_of_base_similarity_measure_csv_files)

  ######################

  #load all similarity dataframes
  list_of_similarity_dataframes = []
  for file in list_of_paths_in_experiment_output_folder_of_base_similarity_measure_csv_files:
    list_of_similarity_dataframes.append(load_csv_from_local_directory_to_dataframe(file))

  #get list of unique dataframes.
  ##load from query list
  queried_products_sample_dataframe = load_csv_from_local_directory_to_dataframe(
      queried_products_sample_dataframe_csv_path)

  product_dataframe = load_csv_from_local_directory_to_dataframe(sampled_product_dataframe_csv_path)

  num_rows_queried_products_sample_dataframe = queried_products_sample_dataframe.shape[0]
  num_rows_similarity_dataframe = list_of_similarity_dataframes[0].shape[0]
  num_rows_product_dataframe = product_dataframe.shape[0]
  # composite_similarity_and_accuracy_dataframe = load_csv_from_local_directory_to_dataframe(list_of_paths_in_experiment_output_folder_of_base_similarity_measure_csv_files[0])

  composite_similarity_and_accuracy_dictionary = {
      'searched_item_unit_id': [None]*num_rows_similarity_dataframe,
      'similar_item_unit_id': [None]*num_rows_similarity_dataframe,
      'searched_item_query': [None]*num_rows_similarity_dataframe,
      'similar_item_query': [None]*num_rows_similarity_dataframe,
      'searched_item_relevance': [None]*num_rows_similarity_dataframe,
      'similar_item_relevance': [None]*num_rows_similarity_dataframe,
      'composite_similarity_score': [None]*num_rows_similarity_dataframe,
      'accuracy': [None]*num_rows_similarity_dataframe
  }

  # temp_composite_similarity_dictionary_for_single_search_query = {
  #     'searched_item_unit_id': [None]*num_rows_similarity_dataframe,
  #     'similar_item_unit_id': [None]*num_rows_similarity_dataframe,
  #     'searched_item_query': [None]*num_rows_similarity_dataframe,
  #     'similar_item_query': [None]*num_rows_similarity_dataframe,
  #     'searched_item_relevance': [None]*num_rows_similarity_dataframe,
  #     'similar_item_relevance': [None]*num_rows_similarity_dataframe,
  #     'composite_similarity_score': [None]*num_rows_similarity_dataframe
  # }

  composite_similarity_and_accuracy_dataframe = pd.DataFrame(data = composite_similarity_and_accuracy_dictionary)


  #loop through queries lits. do pandas filter by a certain value on the similarity dataframe to get indexes
  for i in range(num_rows_queried_products_sample_dataframe):
    queried_product_id = queried_products_sample_dataframe['_unit_id'][i]
    # ipdb.set_trace()
    
    #for a single query item index, loop through similar itemscreate a dataframe for all the composite similarity 
    #cont: scores for that item. Append to list
    index_list = np.where(list_of_similarity_dataframes[0]["searched_item_unit_id"] == queried_product_id)[0].tolist()
    for j in index_list:
      scores_list_for_particular_index = [dataframe['similarity_score'][j] for dataframe in list_of_similarity_dataframes]
      # ipdb.set_trace()
      composite_similarity_and_accuracy_dataframe['composite_similarity_score'][j] = composite_similarity_function(scores_list_for_particular_index)
    
      searched_item_dataframe_row = product_dataframe.loc[product_dataframe['_unit_id'] == queried_product_id].iloc[0]
      similar_item_product_id = list_of_similarity_dataframes[0]['similar_item_unit_id'][j]
      similar_item_dataframe_row = product_dataframe.loc[product_dataframe['_unit_id'] == similar_item_product_id].iloc[0]

      composite_similarity_and_accuracy_dataframe['searched_item_unit_id'][j] = queried_product_id
      composite_similarity_and_accuracy_dataframe['similar_item_unit_id'][j] = similar_item_product_id
      composite_similarity_and_accuracy_dataframe['searched_item_query'][j] = searched_item_dataframe_row['query']
      composite_similarity_and_accuracy_dataframe['similar_item_query'][j] = similar_item_dataframe_row['query']
      composite_similarity_and_accuracy_dataframe['searched_item_relevance'][j] = searched_item_dataframe_row['relevance']
      composite_similarity_and_accuracy_dataframe['similar_item_relevance'][j] = similar_item_dataframe_row['relevance']
      
    # ipdb.set_trace()
    temp_dataframe_for_single_search_query = composite_similarity_and_accuracy_dataframe.iloc[index_list,:]
    temp_dataframe_for_single_search_query = temp_dataframe_for_single_search_query.sort_values(by='composite_similarity_score', ascending=False).iloc[:num_most_similar_items_for_accuracy]
    accuracy, numMatchingCategory, numNonMatchingCategory = calculate_output_dataframe_accuracy_and_write_in_dataframe(temp_dataframe_for_single_search_query, relevance_threshold)
    # ipdb.set_trace()
    composite_similarity_and_accuracy_dataframe['accuracy'][index_list[0]] = accuracy

  # df = DataFrame
  composite_similarity_and_accuracy_dataframe.to_csv(composite_similarity_dataframe_csv_path, index=False)
  return composite_similarity_dataframe_csv_path
    #create new dataframe from list. Calculate overall accuracy. Dont think i want to sort by similarity score yet
    #append list to overall dataframe

composite_similarity_dataframe_csv_path = create_composite_similarity_dataframe_csv_with_assuming_query_is_available(
  experiment = experiment,
  list_of_files_in_experiment_output_folder_of_base_similarity_measure_csv_files = list_of_files_in_experiment_output_folder_of_base_similarity_measure_csv_files,
  composite_similarity_dataframe_csv_name = composite_similarity_dataframe_csv_name, 
  relevance_threshold = relevance_threshold, 
  num_most_similar_items_for_accuracy = num_most_similar_items_for_accuracy, 
  composite_similarity_function = composite_similarity_function)

experiment.composite_similarity_dataframe_csv_path = composite_similarity_dataframe_csv_path

['experiment_outputs/experiment5/similarity_dataframe_product_title_preprocessed_fasttext_cosine_similarity_.csv', 'experiment_outputs/experiment5/similarity_dataframe_product_description_preprocessed_fasttext_cosine_similarity_.csv']


#Accuracy Measure

##Inputs

In [25]:
###########################################
relevance_threshold = 2



###########################################
num_most_similar_items_for_accuracy = 10



###########################################
accuracy_dataframe_csv_path = experiment.experiment_output_folder_path+'accuracy_dataframe.csv'
experiment.accuracy_dataframe_csv_path = accuracy_dataframe_csv_path

###########################################



##Code

In [26]:
def calculate_accuracy_using_method1(
    experiment,
    relevance_threshold, 
    num_most_similar_items_for_accuracy,
    accuracy_dataframe_csv_path
    ):
  exp = experiment
  #folder path for experiment
  experiment_output_folder_path = exp.experiment_output_folder_path

  queried_products_sample_dataframe_csv_path = exp.queried_products_sample_dataframe_csv_path

  sampled_product_dataframe_csv_path = exp.sampled_product_dataframe_csv_path

  composite_similarity_dataframe_csv_path = exp.composite_similarity_dataframe_csv_path

  list_of_preprocessed_text_fields = exp.list_of_preprocessed_text_fields
  dictionary_of_converters_for_read_csv = exp.dictionary_of_converters_for_read_csv

  #preprocessing file path
  product_dataframe_with_preprocessed_text_csv_path = exp.product_dataframe_with_preprocessed_text_csv_path

  ###########################################

  #Load the composite similarity dataframe
  composite_similarity_and_accuracy_dataframe = load_csv_from_local_directory_to_dataframe(composite_similarity_dataframe_csv_path)



  #load the search queries, and get the list of search queries
  queried_products_sample_dataframe = load_csv_from_local_directory_to_dataframe(
      queried_products_sample_dataframe_csv_path)

  num_rows_queried_products_sample_dataframe = queried_products_sample_dataframe.shape[0]

  #create an accuracy_dataframe with fields, search_item_unit_id, search_item_query, search item accuracy, total average accuracy
  accuracy_dictionary = {
      'searched_item_unit_id': [None]*num_rows_queried_products_sample_dataframe,
      'searched_item_query': [None]*num_rows_queried_products_sample_dataframe,
      'searched_item_accuracy': [None]*num_rows_queried_products_sample_dataframe,
      'total_average_accuracy_for_experiment': [None]*num_rows_queried_products_sample_dataframe
  }
  accuracy_dataframe = pd.DataFrame(data = accuracy_dictionary)


  #for each search query, index it out from the composite similarity dataframe, 
  #and output accuracy into accuracy_dataframe

  for i in range(num_rows_queried_products_sample_dataframe):
    queried_product_id = queried_products_sample_dataframe['_unit_id'][i]
    search_item_query = queried_products_sample_dataframe['query'][i]
    # ipdb.set_trace()
    
    temp_dataframe_for_single_search_query = composite_similarity_and_accuracy_dataframe.loc[composite_similarity_and_accuracy_dataframe['searched_item_unit_id'] == queried_product_id]
    temp_dataframe_for_single_search_query = temp_dataframe_for_single_search_query.sort_values(by='composite_similarity_score', ascending=False).iloc[:num_most_similar_items_for_accuracy]
    accuracy, numMatchingCategory, numNonMatchingCategory = calculate_output_dataframe_accuracy_and_write_in_dataframe(temp_dataframe_for_single_search_query, relevance_threshold)
    # ipdb.set_trace()
    accuracy_dataframe.iloc[i]['searched_item_unit_id']=queried_product_id
    accuracy_dataframe.iloc[i]['searched_item_query']=search_item_query
    accuracy_dataframe.iloc[i]['searched_item_accuracy']=accuracy
  nparray_of_accuracies = accuracy_dataframe['searched_item_accuracy'].to_numpy()
  mean = float(np.mean(nparray_of_accuracies))
  accuracy_dataframe.iloc[0]['total_average_accuracy_for_experiment']=mean
  accuracy_dataframe.to_csv(accuracy_dataframe_csv_path, index=False)


calculate_accuracy_using_method1(
    experiment = experiment,
    relevance_threshold = relevance_threshold, 
    num_most_similar_items_for_accuracy = num_most_similar_items_for_accuracy,
    accuracy_dataframe_csv_path = accuracy_dataframe_csv_path
    )

##Create debug view

##Input

In [27]:
#list of similarity dataframes
#input
list_of_files_in_experiment_output_folder_of_base_similarity_measure_csv_files = \
['similarity_dataframe_product_title_preprocessed_fasttext_cosine_similarity_.csv',
'similarity_dataframe_product_description_preprocessed_fasttext_cosine_similarity_.csv']


###########################################
view_to_debug_similarity_dataframe_csv_path = experiment_output_folder_path+'view_to_debug_similarity_dataframe.csv'

##Code

In [28]:
def create_custom_results_view_for_crowdflower_experiment(
    experiment,
    list_of_files_in_experiment_output_folder_of_base_similarity_measure_csv_files,
    view_to_debug_similarity_dataframe_csv_path
):

  exp = experiment
  
  experiment_output_folder_path = exp.experiment_output_folder_path

  experiment_output_folder_path = exp.experiment_output_folder_path

  queried_products_sample_dataframe_csv_path = exp.queried_products_sample_dataframe_csv_path

  sampled_product_dataframe_csv_path = exp.sampled_product_dataframe_csv_path

  composite_similarity_dataframe_csv_path = exp.composite_similarity_dataframe_csv_path

  list_of_preprocessed_text_fields = exp.list_of_preprocessed_text_fields
  dictionary_of_converters_for_read_csv = exp.dictionary_of_converters_for_read_csv

  #preprocessing file path
  product_dataframe_with_preprocessed_text_csv_path = exp.product_dataframe_with_preprocessed_text_csv_path

  #calculated based on input
  list_of_paths_in_experiment_output_folder_of_base_similarity_measure_csv_files = \
  [experiment_output_folder_path+base_similarity for base_similarity in list_of_files_in_experiment_output_folder_of_base_similarity_measure_csv_files]
  print(list_of_paths_in_experiment_output_folder_of_base_similarity_measure_csv_files)

  #######################################

  #THIS CODE GETS DEBUG LOG ASSOCIATED WITH A SPECIFIC ACCURACY MEASURE

  #Load the composite similarity dataframe
  composite_similarity_dataframe = load_csv_from_local_directory_to_dataframe(composite_similarity_dataframe_csv_path)

  num_rows_composite_similarity_dataframe = composite_similarity_dataframe.shape[0]

  #Load the accuracy dataframe
  accuracy_dataframe = load_csv_from_local_directory_to_dataframe(accuracy_dataframe_csv_path)

  num_rows_accuracy_dataframe = accuracy_dataframe.shape[0]

  product_dataframe_with_preprocessed_text = pd.read_csv(product_dataframe_with_preprocessed_text_csv_path, converters=dictionary_of_converters_for_read_csv)

  num_rows_view_to_debug_similarity_dictionary = num_most_similar_items_for_accuracy*num_rows_accuracy_dataframe

  #create dataframe
  view_to_debug_similarity_dictionary = {
      'unit_id_searched_item': [None]*num_rows_view_to_debug_similarity_dictionary, 
      'title_searched_item': [None]*num_rows_view_to_debug_similarity_dictionary, 
      'query_searched_item': [None]*num_rows_view_to_debug_similarity_dictionary,
      'relevance_searched_item': [None]*num_rows_view_to_debug_similarity_dictionary,
      'description_searched_item': [None]*num_rows_view_to_debug_similarity_dictionary,
      # preprocessing_used+'_searched_item': target_description_preprocesed_list,
      'unit_id_similar_item': [None]*num_rows_view_to_debug_similarity_dictionary,
      'title_similar_item': [None]*num_rows_view_to_debug_similarity_dictionary,
      'query_similar_item': [None]*num_rows_view_to_debug_similarity_dictionary,
      'relevance_similar_item': [None]*num_rows_view_to_debug_similarity_dictionary,
      'description_similar_item': [None]*num_rows_view_to_debug_similarity_dictionary,
      # preprocessing_used+'_searched_item': dataframe_modified[new_field_name][similar_product_index].tolist(),
      'rank': [None]*num_rows_view_to_debug_similarity_dictionary,
      'composite_similarity_score': [None]*num_rows_view_to_debug_similarity_dictionary, 
      'accuracy_for_item': [None]*num_rows_view_to_debug_similarity_dictionary,
      'total_average_accuracy_for_experiment': [None]*num_rows_view_to_debug_similarity_dictionary
      } 
  for preprocessing_field_name in list_of_preprocessed_text_fields:
    searched_preprocessing_field = preprocessing_field_name+'_searched'
    similar_preprocessing_field = preprocessing_field_name+'_similar'
    view_to_debug_similarity_dictionary[searched_preprocessing_field] = [None]*num_rows_view_to_debug_similarity_dictionary
    view_to_debug_similarity_dictionary[similar_preprocessing_field] = [None]*num_rows_view_to_debug_similarity_dictionary

  for base_similarity_measure in list_of_files_in_experiment_output_folder_of_base_similarity_measure_csv_files:
    view_to_debug_similarity_dictionary[base_similarity_measure] = [None]*num_rows_view_to_debug_similarity_dictionary

  base_similarity_list_of_dataframes = []
  for base_similarity_measure_path in list_of_paths_in_experiment_output_folder_of_base_similarity_measure_csv_files:
    base_similarity_list_of_dataframes.append(load_csv_from_local_directory_to_dataframe(base_similarity_measure_path))

  view_to_debug_similarity_dataframe = pd.DataFrame(data = view_to_debug_similarity_dictionary)

  #loop through accuracy dataframe items
  for i in range(num_rows_accuracy_dataframe):
    #get rows from composite_similarity measure corresponding to a searched unit id of the accuracy dataframe
    ##sort by similarity rank and sample the num_most_similar_items_for_accuracy
    unit_id_searched_item = accuracy_dataframe.iloc[i]['searched_item_unit_id']
    row_searched_item = product_dataframe_with_preprocessed_text.loc[product_dataframe_with_preprocessed_text['_unit_id'] == unit_id_searched_item].iloc[0]
    title_searched_item = row_searched_item['product_title']
    query_searched_item = row_searched_item['query']
    relevance_searched_item = row_searched_item['relevance']
    description_searched_item = row_searched_item['product_description']

    composite_similarity_dataframe_for_one_item = composite_similarity_dataframe.loc[composite_similarity_dataframe['searched_item_unit_id'] == unit_id_searched_item]
    composite_similarity_dataframe_for_one_item = composite_similarity_dataframe_for_one_item.sort_values(by='composite_similarity_score', ascending=False).iloc[:num_most_similar_items_for_accuracy]
    
    #loop through each of these rows
    for j in range(composite_similarity_dataframe_for_one_item.shape[0]):
      #get corresponding unit id row for both searched and similar product from product with preprocessed text dataframe and fill the following:
      ##unit_id_searched_item, title_searched_item, query_searched_item, relevance_searched_item, description_searched_item, composite_similarity, preprocessed text
      # ipdb.set_trace()
      unit_id_similar_item = composite_similarity_dataframe_for_one_item.iloc[j]['similar_item_unit_id']
      row_similar_item = product_dataframe_with_preprocessed_text.loc[product_dataframe_with_preprocessed_text['_unit_id'] == unit_id_similar_item].iloc[0]
      title_similar_item = row_similar_item['product_title']
      query_similar_item = row_similar_item['query']
      relevance_similar_item = row_similar_item['relevance']
      description_similar_item = row_similar_item['product_description']

      #create a index for view_to_debug_similarity_dataframe
      index_for_view_to_debug_similarity_dataframe = i*num_most_similar_items_for_accuracy+j

      view_to_debug_similarity_dataframe.iloc[index_for_view_to_debug_similarity_dataframe]['unit_id_searched_item'] = unit_id_searched_item
      view_to_debug_similarity_dataframe.iloc[index_for_view_to_debug_similarity_dataframe]['row_searched_item'] = row_searched_item
      view_to_debug_similarity_dataframe.iloc[index_for_view_to_debug_similarity_dataframe]['title_searched_item'] = title_searched_item
      view_to_debug_similarity_dataframe.iloc[index_for_view_to_debug_similarity_dataframe]['query_searched_item'] = query_searched_item
      view_to_debug_similarity_dataframe.iloc[index_for_view_to_debug_similarity_dataframe]['relevance_searched_item'] = relevance_searched_item
      view_to_debug_similarity_dataframe.iloc[index_for_view_to_debug_similarity_dataframe]['description_searched_item'] = description_searched_item

      view_to_debug_similarity_dataframe.iloc[index_for_view_to_debug_similarity_dataframe]['unit_id_similar_item'] = unit_id_similar_item
      view_to_debug_similarity_dataframe.iloc[index_for_view_to_debug_similarity_dataframe]['row_similar_item'] = row_similar_item
      view_to_debug_similarity_dataframe.iloc[index_for_view_to_debug_similarity_dataframe]['title_similar_item'] = title_similar_item
      view_to_debug_similarity_dataframe.iloc[index_for_view_to_debug_similarity_dataframe]['query_similar_item'] = query_similar_item
      view_to_debug_similarity_dataframe.iloc[index_for_view_to_debug_similarity_dataframe]['relevance_similar_item'] = relevance_similar_item
      view_to_debug_similarity_dataframe.iloc[index_for_view_to_debug_similarity_dataframe]['description_similar_item'] = description_similar_item

      # ipdb.set_trace()

      #initialize the preprocessed text
      for k in range(len(list_of_preprocessed_text_fields)):
        searched_field = list_of_preprocessed_text_fields[k] + '_searched'
        similar_field = list_of_preprocessed_text_fields[k] + '_similar'
        view_to_debug_similarity_dataframe.iloc[index_for_view_to_debug_similarity_dataframe][searched_field] =  row_searched_item[list_of_preprocessed_text_fields[k]]
        view_to_debug_similarity_dataframe.iloc[index_for_view_to_debug_similarity_dataframe][similar_field] =  row_similar_item[list_of_preprocessed_text_fields[k]]

      
      #from same row, get the base_similarities a well
      for k in range(len(list_of_files_in_experiment_output_folder_of_base_similarity_measure_csv_files)):
        base_similarity_measure = list_of_files_in_experiment_output_folder_of_base_similarity_measure_csv_files[k]
        base_similarity_dataframe = base_similarity_list_of_dataframes[k]
        # ipdb.set_trace()
        score = base_similarity_dataframe.loc[(base_similarity_dataframe['searched_item_unit_id'] == unit_id_searched_item) & (base_similarity_dataframe['similar_item_unit_id'] == unit_id_similar_item)].iloc[0]['similarity_score']
        view_to_debug_similarity_dataframe.iloc[index_for_view_to_debug_similarity_dataframe][base_similarity_measure] = score   

      #get composite similarity measure
      # ipdb.set_trace()
      view_to_debug_similarity_dataframe.iloc[index_for_view_to_debug_similarity_dataframe]['composite_similarity_score'] = composite_similarity_dataframe_for_one_item.iloc[j]['composite_similarity_score']

    index_min_of_write_in_view = i*num_most_similar_items_for_accuracy
    index_max_of_write_in_view = (i+1)*num_most_similar_items_for_accuracy

    #fill in the accuracy_for_item
    view_to_debug_similarity_dataframe.loc[index_min_of_write_in_view,'accuracy_for_item'] = accuracy_dataframe.iloc[i]['searched_item_accuracy']

    #create a rank
    view_to_debug_similarity_dataframe.loc[index_min_of_write_in_view:index_max_of_write_in_view,'rank'] = view_to_debug_similarity_dataframe.iloc[index_min_of_write_in_view:index_max_of_write_in_view]['composite_similarity_score'].rank(method='min', ascending = False)
    # ipdb.set_trace()
  #fill in total_average_accuracy
  # ipdb.set_trace()
  view_to_debug_similarity_dataframe.iloc[0]['total_average_accuracy_for_experiment'] = accuracy_dataframe.iloc[0]['total_average_accuracy_for_experiment']
  view_to_debug_similarity_dataframe.to_csv(view_to_debug_similarity_dataframe_csv_path, index=False)

create_custom_results_view_for_crowdflower_experiment(
    experiment = experiment,
    list_of_files_in_experiment_output_folder_of_base_similarity_measure_csv_files = list_of_files_in_experiment_output_folder_of_base_similarity_measure_csv_files,
    view_to_debug_similarity_dataframe_csv_path = view_to_debug_similarity_dataframe_csv_path
)

['experiment_outputs/experiment5/similarity_dataframe_product_title_preprocessed_fasttext_cosine_similarity_.csv', 'experiment_outputs/experiment5/similarity_dataframe_product_description_preprocessed_fasttext_cosine_similarity_.csv']


##########################################################################

##########################################################################

##########################################################################

##########################################################################


MOUNT GOOGLE DRIVE SO YOU CAN ACCESS FILES FROM GOOGLE DRIVE

Pip Install packages, and import general packages

In [7]:

from collections import OrderedDict
import numpy as np
import spacy
from spacy.lang.en.stop_words import STOP_WORDS

nlp = spacy.load('en_core_web_sm')

class TextRank4Keyword():
    """Extract keywords from text"""
    
    def __init__(self):
        self.d = 0.85 # damping coefficient, usually is .85
        self.min_diff = 1e-5 # convergence threshold
        self.steps = 10 # iteration steps
        self.node_weight = None # save keywords and its weight

    
    def set_stopwords(self, stopwords):  
        """Set stop words"""
        for word in STOP_WORDS.union(set(stopwords)):
            lexeme = nlp.vocab[word]
            lexeme.is_stop = True
    
    def sentence_segment(self, doc, candidate_pos, lower):
        """Store those words only in cadidate_pos"""
        sentences = []
        for sent in doc.sents:
            selected_words = []
            for token in sent:
                # Store words only with cadidate POS tag
                if token.pos_ in candidate_pos and token.is_stop is False:
                    if lower is True:
                        selected_words.append(token.text.lower())
                    else:
                        selected_words.append(token.text)
            sentences.append(selected_words)
        return sentences
        
    def get_vocab(self, sentences):
        """Get all tokens"""
        vocab = OrderedDict()
        i = 0
        for sentence in sentences:
            for word in sentence:
                if word not in vocab:
                    vocab[word] = i
                    i += 1
        return vocab
    
    def get_token_pairs(self, window_size, sentences):
        """Build token_pairs from windows in sentences"""
        token_pairs = list()
        for sentence in sentences:
            for i, word in enumerate(sentence):
                for j in range(i+1, i+window_size):
                    if j >= len(sentence):
                        break
                    pair = (word, sentence[j])
                    if pair not in token_pairs:
                        token_pairs.append(pair)
        return token_pairs
        
    def symmetrize(self, a):
        return a + a.T - np.diag(a.diagonal())
    
    def get_matrix(self, vocab, token_pairs):
        """Get normalized matrix"""
        # Build matrix
        vocab_size = len(vocab)
        g = np.zeros((vocab_size, vocab_size), dtype='float')
        for word1, word2 in token_pairs:
            i, j = vocab[word1], vocab[word2]
            g[i][j] = 1
            
        # Get Symmeric matrix
        g = self.symmetrize(g)
        
        # Normalize matrix by column
        norm = np.sum(g, axis=0)
        g_norm = np.divide(g, norm, where=norm!=0) # this is ignore the 0 element in norm
        
        return g_norm

    
    def get_keywords(self, number=10):
        """Print top number keywords"""
        node_weight = OrderedDict(sorted(self.node_weight.items(), key=lambda t: t[1], reverse=True))
        for i, (key, value) in enumerate(node_weight.items()):
            print(key + ' - ' + str(value))
            if i > number:
                break

    def get_list_of_strings_of_the_top_textrank_keywords_in_order_of_textrank(self, number=10):
      list_of_top_keywords = []

      """Print top number keywords"""
      node_weight = OrderedDict(sorted(self.node_weight.items(), key=lambda t: t[1], reverse=True))
      for i, (key, value) in enumerate(node_weight.items()):
          #print(key + ' - ' + str(value))
          list_of_top_keywords.append(key.lower())
          if i > number:
            break
      return list_of_top_keywords       
        
    def analyze(self, text, 
                candidate_pos=['NOUN', 'PROPN'], 
                window_size=4, lower=False, stopwords=list()):
        """Main function to analyze text"""
        
        # Set stop words
        self.set_stopwords(stopwords)
        
        # Pare text by spaCy
        doc = nlp(text)
        
        # Filter sentences
        sentences = self.sentence_segment(doc, candidate_pos, lower) # list of list of words
        
        # Build vocabulary
        vocab = self.get_vocab(sentences)
        
        # Get token_pairs from windows
        token_pairs = self.get_token_pairs(window_size, sentences)
        
        # Get normalized matrix
        g = self.get_matrix(vocab, token_pairs)
        
        # Initionlization for weight(pagerank value)
        pr = np.array([1] * len(vocab))
        
        # Iteration
        previous_pr = 0
        for epoch in range(self.steps):
            pr = (1-self.d) + self.d * np.dot(g, pr)
            if abs(previous_pr - sum(pr))  < self.min_diff:
                break
            else:
                previous_pr = sum(pr)

        # Get weight for each node
        node_weight = dict()
        for word, index in vocab.items():
            node_weight[word] = pr[index]
        
        self.node_weight = node_weight

In [6]:
#download pretrained word embedding model and load it
##download a file
##unzip a file
def download_and_unzip_pretrained_word_embedding(url_to_word_embedding_zip: str, download_destination_folder_with_slash_at_end: str) -> Tuple[bool, str]:
  #download if unzipped file exists already
  #if not, unzip and check if file exists after unzipping
  #if file exists, return 1 for success
  import os.path
  from os import path
  zipped_file_name = os.path.basename(url_to_word_embedding_zip)
  unzipped_file_name = os.path.splitext(zipped_file_name)[0]

  path_of_zipped_word_embedding = download_destination_folder_with_slash_at_end + zipped_file_name
  desired_path_of_unzipped_word_embedding = download_destination_folder_with_slash_at_end + unzipped_file_name
  zip_file_extension = os.path.splitext(url_to_word_embedding_zip)[1][1:]

  if path.exists(desired_path_of_unzipped_word_embedding) == False:
    !wget -c $url_to_word_embedding_zip -P $download_destination_folder_with_slash_at_end
    if zip_file_extension == 'gz':
      !gunzip $path_of_zipped_word_embedding
    elif zip_file_extension == 'zip':
      !unzip $path_of_zipped_word_embedding -d $download_destination_folder_with_slash_at_end
    else:
      print("This function doesnt handle zipped files with extension: ", zip_file_extension)
      return (False, None)
  else:
    print("datasetAlreadyExists")

  if path.exists(desired_path_of_unzipped_word_embedding) == True:
    successful_in_creating_word_embedding = True
  else:
    successful_in_creating_word_embedding = False
  return (successful_in_creating_word_embedding, desired_path_of_unzipped_word_embedding)

##load word embedding
def load_word_embedding_into_KeyedVectors_model(word_embedding_file_path: str) -> gensim.models.keyedvectors.Word2VecKeyedVectors:
  from gensim.models.keyedvectors import KeyedVectors
  model = KeyedVectors.load_word2vec_format(word_embedding_file_path, binary=True)
  return model

def load_word2vec_word_embedding_into_KeyedVectors_model(word_embedding_file_path: str) -> gensim.models.keyedvectors.Word2VecKeyedVectors:
  model = KeyedVectors.load_word2vec_format(word_embedding_file_path, binary=True)
  return model

def load_fasttext_word_embedding_into_KeyedVectors_model(word_embedding_file_path: str) -> gensim.models.keyedvectors.Word2VecKeyedVectors:
  model = FastText.load_fasttext_format(model_path)
  return model

##delete below because not used
# #load dataset
# #extract desired features from dataset
# def load_dataset_into_pandas(url_of_dataset: str, tuple_of_starting_and_ending_exclusive_index_of_sample: tuple) -> pd.core.frame.DataFrame:
#   #Create pandas dataframe on dataset
#   import pandas as pd
#   #https://data.world/crowdflower/ecommerce-search-relevance
#   df = pd.read_csv(url_of_dataset, encoding = "ISO-8859-1")
#   start,end = tuple_of_starting_and_ending_exclusive_index_of_sample
#   df = df.iloc[start:end]
#   return df

def load_csv_from_local_directory_to_dataframe(path_of_csv: str, csv_encoding: str = "ISO-8859-1") -> pd.core.frame.DataFrame:
  import os.path
  from os import path
  import pandas as pd

  product_dataframe = pd.read_csv(path_of_csv, encoding = csv_encoding)
  return product_dataframe

def sample_n_products_from_all_query_fields_in_dataframe(product_dataframe: pd.core.frame.DataFrame, number_of_products_per_query: int, randomly_sample_from_each_query: bool = False) -> pd.core.frame.DataFrame:
  from random import sample  
  list_of_queries_field = product_dataframe['query'].unique().tolist()

  indexes_of_query_products = []
  for i in range(len(list_of_queries_field)):
    if isinstance(pd.Index(product_dataframe['query']).get_loc(list_of_queries_field[i]), int):
      indexes_of_query_products.append(pd.Index(product_dataframe['query']).get_loc(list_of_queries_field[i]))
    else:
      bool_array_indicating_dataframe_indexes_where_query_matches = pd.Index(product_dataframe['query']).get_loc(list_of_queries_field[i])
      list_of_indexes_for_particular_query = [i for i, x in enumerate(bool_array_indicating_dataframe_indexes_where_query_matches) if x]
      if randomly_sample_from_each_query:
        indexes_of_query_products.extend(sample(list_of_indexes_for_particular_query, k = number_of_products_per_query))
      else:
        indexes_of_query_products.extend(list_of_indexes_for_particular_query[0:number_of_products_per_query])
  
  num_rows_dataframe = product_dataframe.shape[0]
  total_indexes_set = set(range(0,num_rows_dataframe))
  indexes_of_query_products_set = set(indexes_of_query_products)
  indexes_of_nonquery_products = list(total_indexes_set.difference(indexes_of_query_products_set))
  
  queried_products_sample_dataframe = product_dataframe.copy()
  queried_products_sample_dataframe = queried_products_sample_dataframe.drop(indexes_of_nonquery_products)
  return queried_products_sample_dataframe 


#preprocess data
class PreprocessData:
  """
  A class used to find the orientation of the robot with respect to a lane over time.
  """
  def __init__(self):
    """
    Create a PreprocessData object
    Parameters:
      - 
    """
    from nltk import download
    from nltk import word_tokenize
    # Import and download stopwords from NLTK.
    from nltk.corpus import stopwords
    
    download('punkt')  # Download data for tokenizer.
    download('stopwords')  # Download stopwords list.

    # Remove stopwords.
    self.stop_words = stopwords.words('english')

  def make_text_lower_case(self, text: str) -> str:
    """
    make text lower case. Returns None if text is empty
    Params:
      - text: string of the input text we want to make lowercase
    Returns:
      - lower_case_text: the input text made lowercase
    """
    return text.lower()

  def tokenize_string_to_list_of_separate_words(self, text_string: str) -> List[str]:
    """
    make text lower case. Returns None if text is empty
    Params:
      - text: string of the input text we want to make lowercase
    Returns:
      - lower_case_text: the input text made lowercase
    """
    from nltk import word_tokenize
    return word_tokenize(text_string) 

  def remove_stopwords_from_text(self, text_list: List[str]) -> List[str]:
    """
    make text lower case. Returns None if text is empty
    Params:
      - text: string of the input text we want to make lowercase
    Returns:
      - lower_case_text: the input text made lowercase
    """
    return [w for w in text_list if not w in self.stop_words]

  def remove_numbers_and_punctuation_from_text(self, text_list: List[str]) -> List[str]:
    """
    make text lower case. Returns None if text is empty
    Params:
      - text: string of the input text we want to make lowercase
    Returns:
      - lower_case_text: the input text made lowercase
    """
    return [w for w in text_list if w.isalpha()]

  def text_preprocessing_for_WMD(self, text: str) -> List[str]:
    """
    make text lower case. Returns None if text is empty
    Params:
      - text: string of the input text we want to make lowercase
    Returns:
      - lower_case_text: the input text made lowercase
    """
    text = self.make_text_lower_case(text)
    text_list = self.tokenize_string_to_list_of_separate_words(text)
    text_list = self.remove_stopwords_from_text(text_list)
    text_list = self.remove_numbers_and_punctuation_from_text(text_list)
    return text_list

  def text_preprocessing_with_textrank_for_WMD(self, text: str) -> List[str]:
    """
    make text lower case. Returns None if text is empty
    Params:
      - text: string of the input text we want to make lowercase
    Returns:
      - lower_case_text: the input text made lowercase
    """
    text = self.make_text_lower_case(text)
    
    tr4w = TextRank4Keyword()
    tr4w.analyze(text, candidate_pos = ['NOUN', 'PROPN', 'VERB'], window_size=4, lower=False)
    text_list = tr4w.get_list_of_strings_of_the_top_textrank_keywords_in_order_of_textrank(10)

    # text_list = self.tokenize_string_to_list_of_separate_words(text)
    text_list = self.remove_stopwords_from_text(text_list)
    text_list = self.remove_numbers_and_punctuation_from_text(text_list)
    return text_list

  def create_new_field_with_preprocessed_text_in_dataframe(self, dataframe: pd.core.frame.DataFrame, field_to_preprocess: str, new_field_name: str) -> pd.core.frame.DataFrame:
    """
    make text lower case. Returns None if text is empty
    Params:
      - text: string of the input text we want to make lowercase
    Returns:
      - lower_case_text: the input text made lowercase
    """
    num_rows_dataframe = len(dataframe.index)
    preprocessed_text_list = [np.nan] * num_rows_dataframe
    empty_text = []
    for ind in dataframe.index:
      field_text = dataframe[field_to_preprocess][ind]
      #check if field is empty or nan
      if field_text == field_text:
        preprocessed_text = self.text_preprocessing_for_WMD(field_text)
        # preprocessed_text = self.text_preprocessing_with_textrank_for_WMD(field_text)
        if preprocessed_text != []:
          preprocessed_text_list[ind] = preprocessed_text
        else:
          preprocessed_text_list[ind] = empty_text
      else:
        preprocessed_text_list[ind] = empty_text

    dataframe.insert(len(dataframe.columns), new_field_name, preprocessed_text_list, True)
    return dataframe
    
  def create_new_field_with_preprocessed_text_from_multiple_fields_in_dataframe(self, dataframe: pd.core.frame.DataFrame, list_of_fields_to_preprocess_in_order_of_appending: List[str], new_field_name: str) -> pd.core.frame.DataFrame:
    """
    make text lower case. Returns None if text is empty
    Params:
      - text: string of the input text we want to make lowercase
    Returns:
      - lower_case_text: the input text made lowercase
    """
    num_rows_dataframe = len(dataframe.index)
    preprocessed_text_list = [np.nan] * num_rows_dataframe
    empty_text = []

    for ind in dataframe.index:
      list_of_field_text = []
      for i in range(len(list_of_fields_to_preprocess_in_order_of_appending)):
        if dataframe[list_of_fields_to_preprocess_in_order_of_appending[i]][ind] == dataframe[list_of_fields_to_preprocess_in_order_of_appending[i]][ind]:
          list_of_field_text.append(dataframe[list_of_fields_to_preprocess_in_order_of_appending[i]][ind])
      field_text = " ".join(filter(None, list_of_field_text))
      #check if field is empty or nan
      if field_text == field_text:
        preprocessed_text = self.text_preprocessing_for_WMD(field_text)
        # preprocessed_text = self.text_preprocessing_with_textrank_for_WMD(field_text)
        if preprocessed_text != []:
          preprocessed_text_list[ind] = preprocessed_text
        else:
          preprocessed_text_list[ind] = empty_text
      else:
        preprocessed_text_list[ind] = empty_text

    dataframe.insert(len(dataframe.columns), new_field_name, preprocessed_text_list, True)
    return dataframe     

  def create_new_field_in_dataframe_with_preprocessed_text_from_list_of_existing_fields(self, dataframe: pd.core.frame.DataFrame, list_of_fields_to_preprocess_in_order_of_appending: List[str], new_field_name: str, preprocessing_function: Callable[[str],List[str]]) -> pd.core.frame.DataFrame:
    """
    make text lower case. Returns None if text is empty
    Params:
      - text: string of the input text we want to make lowercase
    Returns:
      - lower_case_text: the input text made lowercase
    """
    num_rows_dataframe = len(dataframe.index)
    preprocessed_text_list = [np.nan] * num_rows_dataframe
    empty_text = []

    for ind in dataframe.index:
      list_of_field_text = []
      for i in range(len(list_of_fields_to_preprocess_in_order_of_appending)):
        if dataframe[list_of_fields_to_preprocess_in_order_of_appending[i]][ind] == dataframe[list_of_fields_to_preprocess_in_order_of_appending[i]][ind]:
          list_of_field_text.append(dataframe[list_of_fields_to_preprocess_in_order_of_appending[i]][ind])
      field_text = " ".join(filter(None, list_of_field_text))
      #check if field is empty or nan
      if field_text == field_text:
        preprocessed_text = preprocessing_function(field_text)
        # preprocessed_text = self.text_preprocessing_for_WMD(field_text)
        # preprocessed_text = self.text_preprocessing_with_textrank_for_WMD(field_text)
        if preprocessed_text != []:
          preprocessed_text_list[ind] = preprocessed_text
        else:
          preprocessed_text_list[ind] = empty_text
      else:
        preprocessed_text_list[ind] = empty_text

    dataframe.insert(len(dataframe.columns), new_field_name, preprocessed_text_list, True)
    return dataframe

#find WMD similarity
class WordMoversDistanceSimilarity:
  """
  A class used to find the orientation of the robot with respect to a lane over time.
  """
  def __init__(self, keyed_vector_model: gensim.models.keyedvectors.Word2VecKeyedVectors, list_of_document_lists: List[List[str]], num_similar_queries: int):
    """
    Create a PreprocessData object
    Parameters:
      - 
    """
    self.model = keyed_vector_model
    self.wmd_corpus = list_of_document_lists
    self.num_similar_queries = num_similar_queries
    self.preprocess_data_object = PreprocessData()
    self.similarityMatrix = 0
    self.similar_document_tuple_of_indexes_and_similarity_scores = 0

  def create_similarity_matrix(self):
    """
    make text lower case. Returns None if text is empty
    Params:
      - text: string of the input text we want to make lowercase
    Returns:
      - lower_case_text: the input text made lowercase
    """
    from gensim.similarities import WmdSimilarity
    self.similarityMatrix = WmdSimilarity(self.wmd_corpus, self.model, self.num_similar_queries)

  def find_similar_document_index_and_similarity(self, query_text: str) -> List[tuple]:
    query_text = preprocess(source_doc)
    self.similar_document_tuple_of_indexes_and_similarity_scores = instance[query]  # A query is simply a "look-up" in the similarity class.
    return self.similar_document_tuple_of_indexes_and_similarity_scores

class PrintingResultAndCalculateLoss:
  """
  A class used to find the orientation of the robot with respect to a lane over time.
  """
  def __init__(self, similar_document_tuple_of_indexes_and_similarity_scores: int, data_frame: pd.core.frame.DataFrame):
    """
    Create a PreprocessData object
    Parameters:
      - 
    """
    self.similar_document_tuple_of_indexes_and_similarity_scores = similar_document_tuple_of_indexes_and_similarity_scores
    self.data_frame = data_frame

  def calculate_accuracy_of_similarity_prediction_for_a_query_text_from_actual_(index_of_query_product, score_cutoff):
    """
    Calculate accuracy for a given single similar products query. This query text must be the actual data from the table, so that it will have a 
    label "query" field and "relevance" field.

    if index_of_query_product "relevance" field < score_cutoff: #we dont want to use query items in which the similarity to the search query was low.
      return accuracy = None
    else: 
      for every index in similar_document_tuple_of_indexes_and_similarity_scores:
        number_of_correct_similar_products += (index_of_query_product "query" field == similar_index "query" field)*(similar_index "relevance" field >= score_cutoff)
    accuracy = number_of_correct_similar_products/num_similar_queries

    Parameters:
      - 
    """
  #in order to be classified correctly, the similar product has to be in same search query as target category
  #and has to have a relavance score >= score cutoff.
  #I know this is flawed because there might be a product in the wrong search query but is still similar to the target product.
  #but overall, im hoping this case to be less

  #
    num_similar_queries = len(index_of_query_product)

    search_query = df['query'][sims[i][0]]
    for i in range(num_similar_queries):
      index_of_

def calculate_output_dataframe_accuracy_and_write_in_dataframe(output_dataframe, relevance_threshold):
  numRows = output_dataframe["searched_item_unit_id"].shape[0]
  numMatchingCategory = 0
  numNonMatchingCategory = 0
  # ipdb.set_trace()
  if output_dataframe.iloc[0]["searched_item_relevance"] < relevance_threshold:
    accuracy, numMatchingCategory, numNonMatchingCategory = (-1,-1,-1)
    return accuracy, numMatchingCategory, numNonMatchingCategory
  for i in range(numRows):  
    if output_dataframe.iloc[0]["searched_item_query"] == output_dataframe.iloc[i]["similar_item_query"]:
      numMatchingCategory += 1
    else:
      numNonMatchingCategory +=1
  # print("numMatchingCategory = ", numMatchingCategory)
  # print("numNonMatchingCategory = ", numNonMatchingCategory)
  ##ipdb.set_trace()
  accuracy = numMatchingCategory/numRows
  # output_dataframe["WMDAccuracy"][0] = accuracy
  return accuracy, numMatchingCategory, numNonMatchingCategory



In [5]:
import numpy as np


class DocSim:
    def __init__(self, w2v_model, stopwords=None):
        self.w2v_model = w2v_model
        self.stopwords = stopwords if stopwords is not None else []

    def vectorize(self, doc: str) -> np.ndarray:
        """
        Identify the vector values for each word in the given document
        :param doc:
        :return:
        """
        doc = doc.lower()
        words = [w for w in doc.split(" ") if w not in self.stopwords]
        word_vecs = []
        for word in words:
            try:
                vec = self.w2v_model[word]
                word_vecs.append(vec)
            except KeyError:
                # Ignore, if the word doesn't exist in the vocabulary
                pass

        # Assuming that document vector is the mean of all the word vectors
        # PS: There are other & better ways to do it.
        vector = np.mean(word_vecs, axis=0)
        return vector

    def vectorize_on_list(self, doc) -> np.ndarray:
        """
        Identify the vector values for each word in the given document
        :param doc:
        :return:
        """
        #doc = doc.lower()
        #words = [w for w in doc.split(" ") if w not in self.stopwords]
        words = doc
        
        word_vecs = []
        for word in words:
            try:
                vec = self.w2v_model[word]
                word_vecs.append(vec)
            except KeyError:
                # Ignore, if the word doesn't exist in the vocabulary
                pass

        # Assuming that document vector is the mean of all the word vectors
        # PS: There are other & better ways to do it.
        # ipdb.set_trace()
        vector = np.mean(word_vecs, axis=0)
        return vector


    def _cosine_sim(self, vecA, vecB):
        """Find the cosine similarity distance between two vectors."""
        csim = np.dot(vecA, vecB) / (np.linalg.norm(vecA) * np.linalg.norm(vecB))
        if np.isnan(np.sum(csim)):
            return 0
        return csim

    def calculate_similarity(self, source_doc, target_docs=None, threshold=0):
        """Calculates & returns similarity scores between given source document & all
        the target documents."""
        if not target_docs:
            return []

        if isinstance(target_docs, str):
            target_docs = [target_docs]

        source_vec = self.vectorize(source_doc)
        results = []
        for doc in target_docs:
            target_vec = self.vectorize(doc)
            sim_score = self._cosine_sim(source_vec, target_vec)
            if sim_score > threshold:
                results.append({"score": sim_score, "doc": doc})
            # Sort results by score in desc order
            results.sort(key=lambda k: k["score"], reverse=True)

        return results

    def calculate_similarity_mod(self, source_doc, target_docs=None, threshold=0):
        """Calculates & returns similarity scores between given source document & all
        the target documents."""
        if not target_docs:
            return []

        if isinstance(target_docs, str):
            target_docs = [target_docs]

        source_vec = self.vectorize_on_list(source_doc)
        results = []
        # ipdb.set_trace()
        target_docs = [target_docs]
        for doc in target_docs:
            target_vec = self.vectorize_on_list(doc)
            sim_score = self._cosine_sim(source_vec, target_vec)
            if sim_score > threshold:
                results.append({"score": sim_score, "doc": doc})
            # Sort results by score in desc order
            results.sort(key=lambda k: k["score"], reverse=True)

        return results